These brochure insurance pdf i got from google search engine

In [1]:
!pip install langchain-community
!pip install tiktoken
!pip install faiss-cpu
!pip install PyPDF2  # Install PyPDF2 for PDF handling
!pip install camelot-py[base] # Install camelot for tabular data extraction
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[base] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.9 MB/s eta 0:00:00


In [2]:
import PyPDF2

# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return text

# Extract text from the available PDF files
bank_rakyat = extract_text_from_pdf('/content/PDS+-+Takaful+Amani+Plus+(ENG).pdf')
rhb = extract_text_from_pdf('/content/Rahmah_PDS.pdf')
hong_leong = extract_text_from_pdf('/content/hlb-smart-protect-plus-insurance-brochure-bm.pdf')

In [3]:
import re

# Function to split text into chunks
def split_text_into_chunks(text, chunk_size=500):
    # Split text into words
    words = re.findall(r'\S+', text)
    # Create chunks
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

# Use the extracted text from the PDFs
texts = [bank_rakyat,rhb, hong_leong]

# Split each text into chunks
chunks = []
for text in texts:
    chunks.extend(split_text_into_chunks(text))

# Display the first few chunks to verify
chunks[:5]

['1 TAKAFUL AMANI PLUS (GTT) _PDS (ENG) _v0.14 PRODUCT DISCLOSURE SHEET (Read this Product Disclosure Sheet before you decide to participate in the Takaful Amani Plus . Be sure to also read the general terms and conditions.) PRODUCT: TAKAFUL AMANI PLUS UNDERWRITTEN BY: TAKAFUL IKHLAS FAMILY BERHAD Licensed under Islamic Financial Services Act 2013 and regulated by Bank Negara Malaysia Date : 29/11/2019 1) What is this product about? This plan is introduced by Bank Rakyat exclusively for the bank’s customers to provide financial security in the event of death / disability due to natural causes . This is a yearly renewable Group Term Takaful (GTT) product with 3 plans choices . 2) What are the Shariah concepts applicable? Wakalah Refers to a contract where a party, as principal (muwakkil) authorizes another party as his agent ( wakil ) to perform a particular task on matters that may be delegated, with or without imposition of a fee. In the context of the Company , we are appointed as an

In [4]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document # Import Document class
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = '#'

# Convert chunks (strings) to Document objects
documents = [Document(page_content=chunk) for chunk in chunks]

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
# Insert question & answer csv file
db = FAISS.from_documents(documents, OpenAIEmbeddings()) # Use documents instead of chunks

<ipython-input-4-97e9673af94a>:18: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(documents, OpenAIEmbeddings()) # Use documents instead of chunks


In [5]:
query = "The total Gross Takaful Contribution that you have to pay may vary based on the choice of plans as stated below: Plans PLAN A PLAN B PLAN C Total Annual Takaful Contribution RM101.00 RM 6 9.00 RM 3 7.00"
docs = db.similarity_search(query)
print(docs[0].page_content)

Death Expense benefit. 3. The list of 36 critical illness covered are as below: List of 36 Critical Illnesses 1 Stroke – resulting in permanent neurological deficit with persisting clinical symptoms 19 Major Organ / Bone Marrow Transplant 2 TAKAFUL AMANI PLUS (GTT) _PDS (ENG) _v0.14 2 Heart Attack – of specified severity 20 Loss of Speech 3 Kidney Failure – requiring dialysis or kidney transplant 21 Brain Surgery 4 Cancer – of specified severity and does not cover very early cancers 22 Heart Valve Surgery 5 Coronary Artery By -Pass Surgery 23 Loss of Independent Existence 6 Serious Coronary Artery Disease 24 Bacterial Meningitis - resulting in permanent inability to perform Activities of Daily Living 7 Angioplasty and other invasive treatments for coronary artery disease 25 Major Head Trauma - resulting in permanent inability to perform Activities of Daily Living 8 End-Stage Liver Failure 26 Chronic Aplastic Anemia - resulting in permanent Bone Marrow Failure 9 Fulminant Viral Hepatiti

In [6]:
print(documents[1])

page_content='Death Expense benefit. 3. The list of 36 critical illness covered are as below: List of 36 Critical Illnesses 1 Stroke – resulting in permanent neurological deficit with persisting clinical symptoms 19 Major Organ / Bone Marrow Transplant 2 TAKAFUL AMANI PLUS (GTT) _PDS (ENG) _v0.14 2 Heart Attack – of specified severity 20 Loss of Speech 3 Kidney Failure – requiring dialysis or kidney transplant 21 Brain Surgery 4 Cancer – of specified severity and does not cover very early cancers 22 Heart Valve Surgery 5 Coronary Artery By -Pass Surgery 23 Loss of Independent Existence 6 Serious Coronary Artery Disease 24 Bacterial Meningitis - resulting in permanent inability to perform Activities of Daily Living 7 Angioplasty and other invasive treatments for coronary artery disease 25 Major Head Trauma - resulting in permanent inability to perform Activities of Daily Living 8 End-Stage Liver Failure 26 Chronic Aplastic Anemia - resulting in permanent Bone Marrow Failure 9 Fulminant 

In [9]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chains.llm import LLMChain # Import LLMChain
from langchain.prompts import PromptTemplate # Import PromptTemplate

retriever = db.as_retriever()

# Create a custom prompt template with a reduced context window
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT) # Chain with custom prompt

<ipython-input-9-54932caa22e7>:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=PROMPT) # Chain with custom prompt


In [14]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "Which insurance package price plan from 3 banks mention is recommended for individuals with a monthly income between RM3,500 and RM4,500?"
qa.run(query)

'\nBased on the information provided, it appears that the recommended insurance package price plan for individuals with a monthly income between RM3,500 and RM4,500 is the RHB Insurance Berhad Rahmah PA Insurance. This plan provides coverage for death, permanent disablement, and daily hospital cash income, with a minimum entry age of 16 years old and a maximum age of 65 years old. '


Based on the information provided, it appears that the recommended insurance package price plan for individuals with a monthly income between RM3,500 and RM4,500 is the RHB Insurance Berhad Rahmah PA Insurance. This plan provides coverage for death, permanent disablement, and daily hospital cash income, with a minimum entry age of 16 years old and a maximum age of 65 years old.

In [15]:
query = "How much monthly payment for the RHB Insurance Berhad Rahmah PA Insurance?"
qa.run(query)

' The monthly payment for the RHB Insurance Berhad Rahmah PA Insurance varies depending on the plan chosen. The annual premium for Plan 1 is RM9.90, while the annual premium for Plan 2 is RM25.00. The monthly payment would be calculated by dividing the annual premium by 12. Therefore, for Plan 1, the monthly payment would be approximately RM0.83, and for Plan 2, it would be approximately RM2.08.'

In [16]:
query = "What is the maximum age for approval?"
qa.run(query)

' The maximum age for approval is 60 years old (age on next birthday).'

In [17]:
query = "What are the career requirements to be eligible for this insurance?"
qa.run(query)

' The policy does not specify any career requirements for eligibility. The only requirement mentioned is that the insured person must be at least 16 years old and not more than 65 years old. '

In [18]:
query = "For each insurance plan offered by Bank Rakyat, RHB, and Hong Leong, provide a summary of the coverage benefits and their associated costs or premiums"
qa.run(query)

' Bank Rakyat:\n1. Personal Accident Takaful Plan\n- Provides coverage for accidental death, permanent disability, and hospitalization expenses due to accidents\n- Minimum entry age of 18 years old up to 60 years old\n- Premiums start from RM10 per annum\n\n2. Homeowner Takaful Plan\n- Provides coverage for loss or damage to your home and its contents due to fire, flood, or theft\n- Minimum entry age of 18 years old up to 60 years old\n- Premiums start from RM50 per annum\n\n3. Motor Takaful Plan\n- Provides coverage for loss or damage to your vehicle due to accidents, fire, or theft\n- Minimum entry age of 18 years old up to 65 years old\n- Premiums start from RM350 per annum\n\nRHB:\n1. Rahmah PA Insurance\n- Personal accident plan that provides indemnity for death or permanent disablement due to accidents, daily hospital cash income at government hospitals, and bereavement allowance\n- Minimum entry age of 16 years old up to 65 years old\n- Premiums start from RM9.90 per annum\n\n2.

 Bank Rakyat:
1. Personal Accident Takaful Plan
- Provides coverage for accidental death, permanent disability, and hospitalization expenses due to accidents
- Minimum entry age of 18 years old up to 60 years old
- Premiums start from RM10 per annum

2. Homeowner Takaful Plan
- Provides coverage for loss or damage to your home and its contents due to fire, flood, or theft
- Minimum entry age of 18 years old up to 60 years old
- Premiums start from RM50 per annum

3. Motor Takaful Plan
- Provides coverage for loss or damage to your vehicle due to accidents, fire, or theft
- Minimum entry age of 18 years old up to 65 years old
- Premiums start from RM350 per annum

RHB:
1. Rahmah PA Insurance
- Personal accident plan that provides indemnity for death or permanent disablement due to accidents, daily hospital cash income at government hospitals, and bereavement allowance
- Minimum entry age of 16 years old up to 65 years old
- Premiums start from RM9.90 per annum

2. Hong Leong SMART Protect Plus Insurance
- Investment-linked plan that provides life protection

In [19]:
query = "How to apply this insurance?"
qa.run(query)

' To apply for this insurance, you can contact RHB Insurance Bhd at their designated email address or Merimen Client Portal link. Alternatively, you can also contact their intermediary or nearest branches for further assistance.'